In [2]:
import pandas as pd
import numpy as np

In [3]:
sensors = pd.read_csv('../data/01_raw/metadata/wind_turbine_data.csv', sep=';',  header = 0, names= ['id', 'power', 'commissioning_date', 'hub_height', 'diameter', 'nuts_id', 'lon', 'lat', 'NUTS1'], usecols= ['power', 'commissioning_date', 'hub_height', 'diameter', 'nuts_id', 'lon', 'lat'], parse_dates= ['commissioning_date'], infer_datetime_format= True)

display(sensors.head()); display(sensors.shape)

,power,commissioning_date,hub_height,diameter,nuts_id,lon,lat
0,1500.0,2002-01-12,61.50,77.0,DE145,9.628233,48.532278
1,1500.0,2002-01-12,61.50,77.0,DE145,9.636193,48.532907
2,750.0,1999-07-07,70.00,48.0,DE145,9.630243,48.391163
3,750.0,1999-01-07,70.00,48.0,DE145,9.630052,48.392745
4,2000.0,2012-11-23,138.38,82.0,DE145,9.759196,48.448638


(24861, 7)

In [4]:
sensors.loc[(sensors['nuts_id']=='DEF0C'), ['commissioning_date', 'power']].head()

,commissioning_date,power
23293,2017-03-13,3050.0
23294,2017-03-27,3050.0
23295,2016-12-13,3400.0
23296,2005-01-30,1500.0
23297,2015-12-15,3000.0


In [5]:
sensors.loc[(sensors['nuts_id']=='DEF0C') & (sensors['commissioning_date']=='2016-12-15'), ['commissioning_date', 'power']]

,commissioning_date,power
23455,2016-12-15,3400.0


In [6]:
sensors_datesorted = sensors.sort_values('commissioning_date')

display(sensors_datesorted.head())
display(sensors_datesorted.shape)

,power,commissioning_date,hub_height,diameter,nuts_id,lon,lat
23310,500.0,1990-01-11,33.000000,31.000000,DEF0C,9.398496,54.592251
22884,300.0,1990-11-14,33.779783,31.055714,DEF08,10.523799,54.152187
14611,280.0,1991-01-30,35.600000,32.000000,DEA43,8.726981,52.145711
22392,300.0,1991-04-12,33.744016,30.915301,DEF07,8.864479,54.465240
22393,300.0,1991-04-12,33.752516,30.915301,DEF07,8.865694,54.466482


(24861, 7)

In [7]:
deltas_daily_bydistrict = sensors_datesorted.groupby(by=['nuts_id', 'commissioning_date'], sort=False).agg({'power': 'sum'})

display(deltas_daily_bydistrict.head())
display(deltas_daily_bydistrict.shape)
display(deltas_daily_bydistrict.index)
display( deltas_daily_bydistrict.loc[('DEF0C',),] )

,,power
nuts_id,commissioning_date,
DEF0C,1990-01-11,500.0
DEF08,1990-11-14,300.0
DEA43,1991-01-30,280.0
DEF07,1991-04-12,900.0
DEA5B,1991-11-08,500.0


(13591, 1)

MultiIndex([('DEF0C', '1990-01-11'),
            ('DEF08', '1990-11-14'),
            ('DEA43', '1991-01-30'),
            ('DEF07', '1991-04-12'),
            ('DEA5B', '1991-11-08'),
            ('DEF0C', '1991-11-25'),
            ('DEA34', '1992-02-14'),
            ('DEF07', '1992-02-20'),
            ('DEA43', '1992-04-08'),
            ('DEF07', '1992-07-10'),
            ...
            ('DEB3H', '2017-09-01'),
            ('DE724', '2017-09-02'),
            ('DEF0D', '2017-09-02'),
            ('DE11D', '2017-09-03'),
            ('DEE0D', '2017-10-01'),
            ('DEE0D', '2017-11-01'),
            ('DE724', '2017-11-01'),
            ('DE11D', '2017-11-01'),
            ('DEB15', '2017-11-03'),
            ('DEB3H', '2017-12-01')],
           names=['nuts_id', 'commissioning_date'], length=13591)

,power
commissioning_date,
1990-01-11,500.0
1991-11-25,300.0
1994-01-07,600.0
1994-01-10,600.0
1994-01-11,500.0
...,...
2016-09-28,3000.0
2016-12-13,3400.0
2016-12-15,3400.0


In [8]:
installed_power = deltas_daily_bydistrict.groupby('nuts_id').agg({'power': 'cumsum'})

display(installed_power.head())
display(installed_power.shape) 
display( installed_power.loc[('DEF0C',),] )

,,power
nuts_id,commissioning_date,
DEF0C,1990-01-11,500.0
DEF08,1990-11-14,300.0
DEA43,1991-01-30,280.0
DEF07,1991-04-12,900.0
DEA5B,1991-11-08,500.0


(13591, 1)

,power
commissioning_date,
1990-01-11,500.000000
1991-11-25,800.000000
1994-01-07,1400.000000
1994-01-10,2000.000000
1994-01-11,2500.000000
...,...
2016-09-28,843051.770583
2016-12-13,846451.770583
2016-12-15,849851.770583


In [9]:
display(deltas_daily_bydistrict.loc[('DEF0C', pd.Timestamp('1994-01-11')),] == installed_power.loc[('DEF0C', pd.Timestamp('1994-01-11')),] - installed_power.loc[('DEF0C', pd.Timestamp('1994-01-10')),])

display(deltas_daily_bydistrict.loc[('DEF0C', pd.Timestamp('1994-01-11')),])

display(installed_power.loc[('DEF0C', pd.Timestamp('1994-01-11')),] - installed_power.loc[('DEF0C', pd.Timestamp('1994-01-10')),])

power    True
dtype: bool

power    500.0
Name: (DEF0C, 1994-01-11 00:00:00), dtype: float64

power    500.0
dtype: float64

In [10]:
installed_power_unstacked = installed_power.unstack(level='nuts_id')

display(installed_power_unstacked.head(3))
display(installed_power_unstacked.iloc[200:303,:])
display(installed_power_unstacked.shape)

power                                                    \
nuts_id             DEF0C  DEF08  DEA43 DEF07 DEA5B DEA34 DEF0B DEE0E DE40F   
commissioning_date                                                            
1990-01-11          500.0    NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1990-11-14            NaN  300.0    NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1991-01-30            NaN    NaN  280.0   NaN   NaN   NaN   NaN   NaN   NaN   

                          ...                                                  \
nuts_id            DEF05  ... DE21C DE251 DE943 DE24C DE266 DEA2C DE221 DE239   
commissioning_date        ...                                                   
1990-01-11           NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1990-11-14           NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1991-01-30           NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

                                
nuts_id            DE21L DE715  
commissioning_date              
1990-01-11           NaN   NaN  
1990-11-14           NaN   NaN  
1991-01-30           NaN   NaN  

[3 rows x 303 columns]

power                                                  \
nuts_id            DEF0C    DEF08 DEA43 DEF07    DEA5B DEA34 DEF0B DEE0E   
commissioning_date                                                         
1995-08-25           NaN      NaN   NaN   NaN      NaN   NaN   NaN   NaN   
1995-08-29           NaN  11100.0   NaN   NaN      NaN   NaN   NaN   NaN   
1995-08-30           NaN      NaN   NaN   NaN      NaN   NaN   NaN   NaN   
1995-08-31           NaN      NaN   NaN   NaN      NaN   NaN   NaN   NaN   
1995-09-05           NaN      NaN   NaN   NaN  15600.0   NaN   NaN   NaN   
...                  ...      ...   ...   ...      ...   ...   ...   ...   
1996-04-05           NaN      NaN   NaN   NaN      NaN   NaN   NaN   NaN   
1996-04-07           NaN      NaN   NaN   NaN      NaN   NaN   NaN   NaN   
1996-04-08           NaN      NaN   NaN   NaN      NaN   NaN   NaN   NaN   
1996-04-10           NaN      NaN   NaN   NaN      NaN   NaN   NaN   NaN   
1996-04-15           NaN      NaN   NaN   NaN      NaN   NaN   NaN   NaN   

                                  ...                                      \
nuts_id              DE40F DEF05  ... DE21C DE251 DE943 DE24C DE266 DEA2C   
commissioning_date                ...                                       
1995-08-25             NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
1995-08-29             NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
1995-08-30          3900.0   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
1995-08-31             NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
1995-09-05             NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
...                    ...   ...  ...   ...   ...   ...   ...   ...   ...   
1996-04-05             NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
1996-04-07             NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
1996-04-08             NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
1996-04-10             NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
1996-04-15             NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   

                                            
nuts_id            DE221 DE239 DE21L DE715  
commissioning_date                          
1995-08-25           NaN   NaN   NaN   NaN  
1995-08-29           NaN   NaN   NaN   NaN  
1995-08-30           NaN   NaN   NaN   NaN  
1995-08-31           NaN   NaN   NaN   NaN  
1995-09-05           NaN   NaN   NaN   NaN  
...                  ...   ...   ...   ...  
1996-04-05           NaN   NaN   NaN   NaN  
1996-04-07           NaN   NaN   NaN   NaN  
1996-04-08           NaN   NaN   NaN   NaN  
1996-04-10           NaN   NaN   NaN   NaN  
1996-04-15           NaN   NaN   NaN   NaN  

[103 rows x 303 columns]

(5834, 303)

In [19]:
installed_power_mts = installed_power_unstacked.asfreq('D').ffill().fillna(value=0, axis='rows')

display(installed_power_mts.head())
display(installed_power_mts.tail())
# display(installed_power_mts.loc['2015-03-01':'2015-03-12',:])
display(installed_power_mts.loc['2015-03-01':'2015-03-12',:])
display(installed_power_mts.shape)

power                                                  \
nuts_id             DEF0C DEF08 DEA43 DEF07 DEA5B DEA34 DEF0B DEE0E DE40F   
commissioning_date                                                          
1990-01-11          500.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1990-01-12          500.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1990-01-13          500.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1990-01-14          500.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1990-01-15          500.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

                          ...                                                  \
nuts_id            DEF05  ... DE21C DE251 DE943 DE24C DE266 DEA2C DE221 DE239   
commissioning_date        ...                                                   
1990-01-11           0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1990-01-12           0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1990-01-13           0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1990-01-14           0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1990-01-15           0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

                                
nuts_id            DE21L DE715  
commissioning_date              
1990-01-11           0.0   0.0  
1990-01-12           0.0   0.0  
1990-01-13           0.0   0.0  
1990-01-14           0.0   0.0  
1990-01-15           0.0   0.0  

[5 rows x 303 columns]

power                                          \
nuts_id                     DEF0C     DEF08    DEA43      DEF07     DEA5B   
commissioning_date                                                          
2017-11-27          855951.770583  487340.0  21065.0  1772350.0  260360.0   
2017-11-28          855951.770583  487340.0  21065.0  1772350.0  260360.0   
2017-11-29          855951.770583  487340.0  21065.0  1772350.0  260360.0   
2017-11-30          855951.770583  487340.0  21065.0  1772350.0  260360.0   
2017-12-01          855951.770583  487340.0  21065.0  1772350.0  260360.0   

                                                                       ...  \
nuts_id                DEA34     DEF0B     DEE0E     DE40F      DEF05  ...   
commissioning_date                                                     ...   
2017-11-27          266725.0  357470.0  228175.0  881550.0  1714470.0  ...   
2017-11-28          266725.0  357470.0  228175.0  881550.0  1714470.0  ...   
2017-11-29          266725.0  357470.0  228175.0  881550.0  1714470.0  ...   
2017-11-30          266725.0  357470.0  228175.0  881550.0  1714470.0  ...   
2017-12-01          266725.0  357470.0  228175.0  881550.0  1714470.0  ...   

                                                                        \
nuts_id              DE21C    DE251    DE943    DE24C    DE266   DEA2C   
commissioning_date                                                       
2017-11-27          6100.0  11400.0  12000.0  10280.0  13200.0  2400.0   
2017-11-28          6100.0  11400.0  12000.0  10280.0  13200.0  2400.0   
2017-11-29          6100.0  11400.0  12000.0  10280.0  13200.0  2400.0   
2017-11-30          6100.0  11400.0  12000.0  10280.0  13200.0  2400.0   
2017-12-01          6100.0  11400.0  12000.0  10280.0  13200.0  2400.0   

                                                       
nuts_id              DE221    DE239    DE21L    DE715  
commissioning_date                                     
2017-11-27          3300.0  16800.0  12000.0  15000.0  
2017-11-28          3300.0  16800.0  12000.0  15000.0  
2017-11-29          3300.0  16800.0  12000.0  15000.0  
2017-11-30          3300.0  16800.0  12000.0  15000.0  
2017-12-01          3300.0  16800.0  12000.0  15000.0  

[5 rows x 303 columns]

power                                          \
nuts_id                     DEF0C     DEF08    DEA43      DEF07     DEA5B   
commissioning_date                                                          
2015-03-01          652051.770583  447640.0  21065.0  1465280.0  220710.0   
2015-03-02          652051.770583  447640.0  21065.0  1465280.0  220710.0   
2015-03-03          652051.770583  447640.0  21065.0  1465280.0  220710.0   
2015-03-04          652051.770583  447640.0  21065.0  1465280.0  220710.0   
2015-03-05          652051.770583  447640.0  21065.0  1465280.0  220710.0   
2015-03-06          652051.770583  447640.0  21065.0  1465280.0  220710.0   
2015-03-07          652051.770583  447640.0  21065.0  1465280.0  220710.0   
2015-03-08          652051.770583  447640.0  21065.0  1465280.0  220710.0   
2015-03-09          652051.770583  447640.0  21065.0  1468480.0  220710.0   
2015-03-10          652051.770583  447640.0  21065.0  1468480.0  220710.0   
2015-03-11          652051.770583  447640.0  21065.0  1468480.0  220710.0   
2015-03-12          652051.770583  447640.0  21065.0  1468480.0  220710.0   

                                                                       ...  \
nuts_id                DEA34     DEF0B     DEE0E     DE40F      DEF05  ...   
commissioning_date                                                     ...   
2015-03-01          266725.0  187670.0  206175.0  821500.0  1300240.0  ...   
2015-03-02          266725.0  190670.0  206175.0  821500.0  1300240.0  ...   
2015-03-03          266725.0  190670.0  206175.0  821500.0  1300240.0  ...   
2015-03-04          266725.0  190670.0  206175.0  821500.0  1300240.0  ...   
2015-03-05          266725.0  190670.0  206175.0  821500.0  1300240.0  ...   
2015-03-06          266725.0  190670.0  206175.0  821500.0  1300240.0  ...   
2015-03-07          266725.0  190670.0  206175.0  821500.0  1303440.0  ...   
2015-03-08          266725.0  190670.0  206175.0  821500.0  1306490.0  ...   
2015-03-09          266725.0  190670.0  206175.0  821500.0  1312740.0  ...   
2015-03-10          266725.0  190670.0  206175.0  821500.0  1316110.0  ...   
2015-03-11          266725.0  190670.0  206175.0  821500.0  1316110.0  ...   
2015-03-12          266725.0  190670.0  206175.0  821500.0  1319410.0  ...   

                                                                             \
nuts_id              DE21C   DE251    DE943   DE24C DE266 DEA2C DE221 DE239   
commissioning_date                                                            
2015-03-01          6100.0  6600.0  12000.0  7500.0   0.0   0.0   0.0   0.0   
2015-03-02          6100.0  6600.0  12000.0  7500.0   0.0   0.0   0.0   0.0   
2015-03-03          6100.0  6600.0  12000.0  7500.0   0.0   0.0   0.0   0.0   
2015-03-04          6100.0  6600.0  12000.0  7500.0   0.0   0.0   0.0   0.0   
2015-03-05          6100.0  6600.0  12000.0  7500.0   0.0   0.0   0.0   0.0   
2015-03-06          6100.0  6600.0  12000.0  7500.0   0.0   0.0   0.0   0.0   
2015-03-07          6100.0  6600.0  12000.0  7500.0   0.0   0.0   0.0   0.0   
2015-03-08          6100.0  6600.0  12000.0  7500.0   0.0   0.0   0.0   0.0   
2015-03-09          6100.0  6600.0  12000.0  7500.0   0.0   0.0   0.0   0.0   
2015-03-10          6100.0  6600.0  12000.0  7500.0   0.0   0.0   0.0   0.0   
2015-03-11          6100.0  6600.0  12000.0  7500.0   0.0   0.0   0.0   0.0   
2015-03-12          6100.0  6600.0  12000.0  7500.0   0.0   0.0   0.0   0.0   

                                
nuts_id            DE21L DE715  
commissioning_date              
2015-03-01           0.0   0.0  
2015-03-02           0.0   0.0  
2015-03-03           0.0   0.0  
2015-03-04           0.0   0.0  
2015-03-05           0.0   0.0  
2015-03-06           0.0   0.0  
2015-03-07           0.0   0.0  
2015-03-08           0.0   0.0  
2015-03-09           0.0   0.0  
2015-03-10           0.0   0.0  
2015-03-11           0.0   0.0  
2015-03-12           0.0   0.0  

[12 rows x 303 colu

(10187, 303)

In [20]:
installed_power_mts = installed_power_mts.droplevel(level=0, axis='columns')

display( installed_power_mts.head(3) )
display( installed_power_mts.tail(3) )

nuts_id,DEF0C,DEF08,DEA43,DEF07,DEA5B,DEA34,DEF0B,DEE0E,DE40F,DEF05,...,DE21C,DE251,DE943,DE24C,DE266,DEA2C,DE221,DE239,DE21L,DE715
commissioning_date,,,,,,,,,,,,,,,,,,,,,
1990-01-11,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-01-12,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-01-13,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


nuts_id,DEF0C,DEF08,DEA43,DEF07,DEA5B,DEA34,DEF0B,DEE0E,DE40F,DEF05,...,DE21C,DE251,DE943,DE24C,DE266,DEA2C,DE221,DE239,DE21L,DE715
commissioning_date,,,,,,,,,,,,,,,,,,,,,
2017-11-29,855951.770583,487340.0,21065.0,1772350.0,260360.0,266725.0,357470.0,228175.0,881550.0,1714470.0,...,6100.0,11400.0,12000.0,10280.0,13200.0,2400.0,3300.0,16800.0,12000.0,15000.0
2017-11-30,855951.770583,487340.0,21065.0,1772350.0,260360.0,266725.0,357470.0,228175.0,881550.0,1714470.0,...,6100.0,11400.0,12000.0,10280.0,13200.0,2400.0,3300.0,16800.0,12000.0,15000.0
2017-12-01,855951.770583,487340.0,21065.0,1772350.0,260360.0,266725.0,357470.0,228175.0,881550.0,1714470.0,...,6100.0,11400.0,12000.0,10280.0,13200.0,2400.0,3300.0,16800.0,12000.0,15000.0
